We will copy the commands from the Titanic example and apply them to a new data set (c08_data.xlsx)

In [ ]:
import pandas as pd
import numpy as np
import sys
from sklearn.model_selection import train_test_split
load("~/conjecturing/sage/conjecturing.py")

In [ ]:
inv_file = open("2024_02_20_inv.txt", "w")
prop_file = open("2024_02_20_prop.txt", "w")

In [ ]:
num_train = 100
my_skips = 0.3

In [ ]:
my_data =pd.read_excel("c08_data.xlsx",
                    header=int(0),
                    sheet_name = "Sheet1"   
                    )
print(my_data.shape)
my_data.head()

Check the data types of the columns. For categorical data, make sure the type is integer or objects. Make sure the categories do not contain special characters besides numbers and "_".

In [ ]:
my_data.info()

In [ ]:
#"PTSURV_T", "PT_HOS_T"
my_data = my_data.astype({"PTSURV_T": object, "PT_HOS_T": object})


The data has NaNs and missing data. We want to "drop" the rows where our TARGET doesn't have a value.

In [ ]:
my_data.dropna(subset=["PTSURV_T"], inplace=True)
#my_data.dropna().reset_index(drop=True)

we need at least two invariants for our experiment. The "PTSURV_T" categorical variable will be our target. We'll also need at least two categorical (property) variables, and you only see 1 below. But invariant relations will be added to the properties list.

In [ ]:
#invariant_names=["Tx_R_O1", "N_Tx_Ctr1", "N_R_Ctr1", "N_Center", "M_R_O1", "M_R_ST1",   "N_PT_TRANS_S", "PT_TRANS_T1", "N_PTSURV_SUM", "N_PT_HOS_S", "VAVF_F", "N_DP_HGBD12", "N_DP_HGBD", "HOMEHD", "PD", "HD", "TOTSTAS"]
invariant_names=["Weighted_HIE_DSA", "Flagging_ratio", "Tx_R_O1", "N_Tx_Ctr1"]

categorical_names=["PTSURV_T", "PT_HOS_T"]
target = "PTSURV_T"

Generic code starts here. Rename target column. Select columns.

In [ ]:
if target in categorical_names:
    categorical_names[categorical_names.index(target)] = "TARGET"
else:
    invariant_names[invariant_names.index(target)] = "TARGET"

my_data = my_data.rename(columns={target: "TARGET"})
my_data = my_data[invariant_names + categorical_names]
my_data.columns

Convert categorical variables to dummies. One dummy for each binary variable and one dummy for each level for variables with more than two levels.

In [ ]:
property_names = []
for col in categorical_names:
    if col != "TARGET":
        unique_vals=list(my_data[col].unique())  # if nan is a level
        #unique_vals=list(my_data[col].dropna().unique())  # if nan is not a level
        if len(unique_vals)==2: # just use one level for binary features
            property_names.append(col+"_"+str(unique_vals[1]))
        elif len(unique_vals) > 2: #one property for each level.
            for level in unique_vals:
                property_names.append(col+"_"+str(level))


if "TARGET" in categorical_names:
    target_property_names = []
    unique_vals = list(my_data["TARGET"].unique()) # if nan is a level
    #unique_vals = list(my_data["TARGET"].dropna().unique()) # if nan is not a level
    if len(unique_vals)==2:
        target_property_names.append("TARGET_"+str(unique_vals[1]))
    elif len(unique_vals) > 2:
        for level in unique_vals:
            target_property_names.append("TARGET_"+str(level))
            
my_df = pd.get_dummies(my_data, 
                       columns=categorical_names,
                       dtype=np.uint8,
                       dummy_na=True,  # False is the default.  If False, use dropna() above
                       drop_first=False) # False is the default

my_df = my_df.rename(lambda col: col.replace('.0', ''), axis='columns')
my_df.head()




Define class, invariants, properties, and target properties (if applicable).

In [ ]:
class Example():
    def __init__(self, name, mydf):
        self.name = name
        self.mydf = mydf
        
for i in invariant_names:
    inv = build_inv(i)
    setattr(Example,inv.__name__,inv )

for i in property_names:
    prop = build_prop(i)
    setattr(Example, prop.__name__,prop)

if "TARGET" in categorical_names:
    for i in target_property_names:
        prop = build_prop(i)
        setattr(Example, prop.__name__, prop)
else:
    target_invariant = invariant_names.index("TARGET")
print(property_names)

Split into training and testing data.

In [ ]:
if "TARGET" in categorical_names:
    X_train, X_test = train_test_split(
        my_df.index,
        stratify=my_data["TARGET"],  # stratify on target levels
        train_size=num_train,
        random_state=12345
    )
else:
    X_train, X_test = train_test_split(
        my_df.index,
        train_size=num_train,
        random_state=12345
    )

Create examples for conjecturing.

In [ ]:
train_examples = [Example(i, my_df) for i in X_train]
test_examples = [Example(i, my_df) for i in X_test]

Get lists of invariant and property functions.

In [ ]:
invariants =[]
for i in invariant_names:
    invariants.append(Example.__dict__[i])
properties=[]
for i in property_names:
    properties.append(Example.__dict__[i])
target_properties=[]
if "TARGET" in categorical_names:
    for i in target_property_names:
        target_properties.append(Example.__dict__[i])


Invariant conjecturing - upper and lower bounds.

In [ ]:
#define operators for expression tree to build upper bounsand lower bouns for each class
use_operators =  { '-1', '+1', '*2', '/2', '^2', '-()', '1/', 
                  'sqrt', 'ceil', 
                  'floor', 'abs', '+', '*', 'max', 'min', '-', '/'}

inv_conjectures = []

if "TARGET" in categorical_names:
    for value in target_property_names:
        print(value)
        target_property = Example.__dict__[value]
        my_examples = [example for example in train_examples if target_property(example) == True]
        for inv in invariants:
            sys.stdout.flush()
            inv_of_interest = invariants.index(inv)
            conjs = conjecture(my_examples, 
                               invariants, 
                               inv_of_interest, 
                               operators=use_operators, 
                               upperBound=True, 
                               time=Integer(5)
                             # ,debug=True,
                             #  verbose=True,
                               ,skips=my_skips
                              )
            convert_conjecture_names(conjs)
            inv_conjectures += conjs

            conjs = conjecture(my_examples, 
                               invariants, 
                               inv_of_interest, 
                               operators=use_operators, 
                               upperBound=False, 
                               time=Integer(5)
                              ,skips=my_skips)
            convert_conjecture_names(conjs)
            inv_conjectures += conjs
    print(len(inv_conjectures))
    if len(target_property_names) == 1:
        value = target_property_names[0]
        print(value + " False")
        target_property = Example.__dict__[value]
        my_examples = [example for example in train_examples if target_property(example) == False]
        for inv in invariants:
            sys.stdout.flush()
            inv_of_interest = invariants.index(inv)
            conjs = conjecture(my_examples, 
                               invariants, 
                               inv_of_interest, 
                               operators=use_operators, 
                               upperBound=True, 
                               time=Integer(5)
                             # ,debug=True,
                             #  verbose=True,
                               ,skips=my_skips
                              )
            convert_conjecture_names(conjs)
            inv_conjectures += conjs

            conjs = conjecture(my_examples, 
                               invariants, 
                               inv_of_interest, 
                               operators=use_operators, 
                               upperBound=False, 
                               time=Integer(5)
                              ,skips=my_skips)
            convert_conjecture_names(conjs)
            inv_conjectures += conjs
else: # target is an invariant
    my_examples = [example for example in train_examples]
    conjs = conjecture(my_examples, 
                       invariants, 
                       target_invariant, 
                       operators=use_operators, 
                       upperBound=True, 
                       time=Integer(5)
                        # ,debug=True,
                        #  verbose=True,
                        ,skips=my_skips)
    convert_conjecture_names(conjs)
    inv_conjectures += conjs
    conjs = conjecture(my_examples, 
                       invariants, 
                       target_invariant, 
                       operators=use_operators,
                       upperBound=False, 
                       time=Integer(5)
                       ,skips=my_skips)
    convert_conjecture_names(conjs)
    inv_conjectures += conjs     
print(len(inv_conjectures))  

for c in inv_conjectures:
    inv_file.write("%s\n" % c)
    inv_file.flush()
inv_file.close()


Property conjecturing - sufficient conditions for a categorical target values. For a binary target, get sufficient conditions for the positive class and necessary conditions for the negative class.

In [ ]:
all_properties = ["TARGET"] + properties + inv_conjectures #"TARGET" is just a placeholder
prop_conjs = []
conditions = []
if "TARGET" in categorical_names:
    for value in target_property_names:
        print(value)
        all_properties[0] = Example.__dict__[value]
        #print(all_properties)
        these_prop_conjs = propertyBasedConjecture(objects=train_examples, # edit here 6/27/23
                                           properties = all_properties,
                                           mainProperty=0,
                                           #verbose=True,
                                           #debug=True,
                                           skips=my_skips)
        for c in these_prop_conjs: # edit here 6/27/23 just get premises once
            conditions.append(get_premise(c, myprint=False))
        prop_conjs += these_prop_conjs
    if len(target_property_names) == 1:
        print(value + " Necessary")
        all_properties[0] = Example.__dict__[value]
        these_prop_conjs = propertyBasedConjecture(objects=train_examples,  # edit here 6/27/23
                                           properties = all_properties,
                                           mainProperty=0,
                                           sufficient=False,
                                           #verbose=True,
                                            #  debug=True,
                                             skips=my_skips)
        for c in these_prop_conjs:
            conditions.append(get_conclusion(c, myprint=False))
        prop_conjs += these_prop_conjs  # edit here 6/27/23
        
for c in prop_conjs:
    prop_file.write("%s\n" % convert_name_back(c.__name__))
    prop_file.flush()
    
prop_file.close()

In [ ]:
for conj in prop_conjs:
    print(convert_name_back(conj.__name__))